In [2]:
import numpy as np
import pandas as pd
import os.path


In [ ]:
frame_num = list() # создаем пустые массивы, которые будем заполнять номером кадра, на котором возможно высчитать угол и соответственно расчитаный угол
angles = list() 
for i in range(103,7423): # производим итерацию по каждому кадру
    filexists = os.path.isfile(r"C:\Users\alexa\Desktop\labels-20240420T112017Z-001\labels\CNTRL_" + str(i) + ".txt") # проверяем существование файла, так как не для всех кадров есть данные
    if filexists==True:
        df = pd.read_csv(r"C:\Users\alexa\Desktop\labels-20240420T112017Z-001\labels\CNTRL_" + str(i) + ".txt", sep=" ", header=None) # считываем название каждого файла
        sh = df.shape[0] #определяем количество строк, соответственно данных для классов
        
        
        if sh>=3:
                    # создаем массивы данных для каждого класса
            arrLC = []
            arrRC = []
            arrR = []
            
            
            for j in range (sh): # проводим итерации для каждой строчки, с классификацией по классам.
                if df.iloc[j][0] == 0:
                    arrLC.extend(df.loc[[j]].to_numpy())
                if df.iloc[j][0] == 1:
                    arrRC.extend(df.loc[[j]].to_numpy())
                if df.iloc[j][0] == 2:
                    arrR.extend(df.loc[[j]].to_numpy())
            if len(arrR)==0:
                continue
            
            # определяем максимальное расстояние между координатами двух точек  для определения угла
            arrsum = arrLC + arrRC
            if len(arrsum) > 0:
                LC = arrsum[np.argmin(arrsum, axis=0)[1]]
                RC = arrsum[np.argmax(arrsum, axis=0)[1]]
            # считаем координаты точек, получающегося треугольника    
                b = np.array([arrR[0][1]+arrR[0][3]/2,arrR[0][2]+arrR[0][4]/2])
                a = np.array([LC[1]+LC[3]/2,LC[2]+LC[4]/2])
                c = np.array([RC[1]+RC[3]/2,RC[2]+RC[4]/2])
                ba = a - b
                bc = c - b
            #считаем угол необходимый угол B и заносим угол и номер итерации в соответствующий массив
                cos_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
                angle = np.degrees(np.arccos(cos_angle))
                angles.append(angle)
                frame_num.append(i)
            else: # отбрасываем кадры, когда отсутствуют координаты левой и правой стропы
                continue
        else: # отбрасываем кадры, когда нет данных по минимум 3 классам
            continue
    else: # переходим к следующему элементу, если кадра не существует
        continue
angles, frame_num

In [18]:
# создаем DataFrame, для дальнейшего экспорта в формат .txt
angleslst = list(zip(angles, frame_num))
hr_df_1 = pd.DataFrame(angleslst, columns = None)
hr_df_1 

,0,1
0,3.914546,354
1,3.234051,356
2,7.287107,473
3,7.148556,491
4,7.750108,492
...,...,...
3504,0.356181,7116
3505,0.648465,7117
3506,0.918604,7119
3507,0.856898,7238


In [19]:
hr_df_1.to_csv('angles.txt', header = None, index = None, sep = ' ') # создаем итоговый текстовый файл с результатами 